# Step 1: Login as External Researcher

In [1]:
import syft as sy

datasites = list()
for i in range(3):
    server = sy.orchestra.launch(name="fl-datasite-"+str(i)) # connects to same server
    client = server.login(email="sheldon@caltech.edu", password="changethis")
    datasites.append(client)

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-0: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-1: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

SyftInfo: You have launched a development server at http://0.0.0.0:None.It is intended only for local use.

Logged into <fl-datasite-2: High side Datasite> as <sheldon@caltech.edu>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

# Step 2: Train models on all datasites

In [36]:
results = list()

for dasite in datasites:
    print()
    print(dasite)
    images_ptr = dasite.datasets['MNIST Dataset'].assets['images']
    labels_ptr = dasite.datasets['MNIST Dataset'].assets['labels']
    
    results.append(dasite.code.train(mnist_images=images_ptr, 
                                     mnist_labels=labels_ptr).get_from(dasite))


<DatasiteClient: fl-datasite-0>


SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.


<DatasiteClient: fl-datasite-1>


SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.


<DatasiteClient: fl-datasite-2>


SyftWarning: Policy is no longer valid. count: 1 >= limit: 1. Loading results from cache.

# Step 3: Merge models

In [55]:
from itertools import groupby
from collections import OrderedDict

def ave(d):
    _data = sorted([i for b in d for i in b.items()], key=lambda x:x[0])
    _d = [(a, [j for _, j in b]) for a, b in groupby(_data, key=lambda x:x[0])]
    return OrderedDict({a:ave(b) if isinstance(b[0], dict) else sum(b)/float(len(b)) for a, b in _d})


In [59]:
models_weights = list(map(lambda x:x[1], results))

In [60]:
new_model_weights = ave(models_weights)

In [63]:
import torch as torch
from torch import nn
# Define the neural network class
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 10)

    def forward(self, x):
        x = torch.log_softmax(self.fc1(x.view(-1, 784)), dim=1)
        return x

In [70]:
new_model = MLP()
new_model.load_state_dict(new_model_weights)

<All keys matched successfully>